# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [21]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from os.path import dirname

#### Creating list of filepaths to process original event csv data files

In [22]:
# checking the current working directory
print(os.getcwd())

# get the current folder and subfolder event data
filepath = dirname(dirname(os.getcwd())) + '/event_data'

# collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

# check number of files
print(len(file_path_list))

/home/fulu/code/cassandra-data-modeling/src/notebooks
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading the csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skip header row
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [24]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="../../images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [28]:
from cassandra.cluster import Cluster

# make a connection to a Cassandra instance your local machine 
cluster = Cluster(['127.0.0.1'])

# establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [29]:
# create a Keyspace
create_keyspace_query = """
    CREATE KEYSPACE IF NOT EXISTS music_app
    WITH REPLICATION =
    { 'class': 'SimpleStrategy', 'replication_factor' : 1}
"""

try:
    session.execute(create_keyspace_query)
except Exception as e:
    print(e)

#### Set Keyspace

In [30]:
# set keyspace to the one specified above
session.set_keyspace('music_app')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [31]:
# load data from csv to pandas dataframe
file = 'event_datafile_new.csv'
df = pd.read_csv(file, encoding='utf8')

### Query 1 - session_history table
The primary key for the first table will be a compound key consisting of a partition key (session_id) and clustering column (session_item).
Records on one partition will belong to the same session and they will be sorted by session_item number.

In [32]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
query  =  "CREATE TABLE IF NOT EXISTS session_history "
query += "(session_id INT, session_item INT, artist TEXT, song TEXT, length FLOAT, \
           PRIMARY KEY(session_id, session_item))"

try:
    session.execute(query)
except Exception as e:
    print(e)        

In [33]:
# load data to first table
for i, row in df.iterrows():
    query  = "INSERT INTO session_history (session_id, session_item, artist, song, length)"
    query += "VALUES(%s, %s, %s, %s, %s)"
    session.execute(query, (row.sessionId, row.itemInSession, row.artist, row.song, row.length))

In [34]:
query = "SELECT * FROM session_history WHERE session_id=338 AND session_item=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['session_id', 'session_item', 'artist', 'song', 'length']) 

,session_id,session_item,artist,song,length
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


### Query 2 - user_history table
The primary key for the second table will be a compound key consisting of a partition key (user_id) and clustering columns (session_id and session_item).
Records on one partition will belong to the same user and they will be sorted by session_id and then by session_item.

In [35]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

query  =  "CREATE TABLE IF NOT EXISTS user_history "
query += "(user_id INT, session_id INT, session_item INT, artist TEXT, song TEXT, user TEXT, \
           PRIMARY KEY((user_id), session_id, session_item))"

try:
    session.execute(query)
except Exception as e:
    print(e)    

In [36]:
# load data to second table
for i, row in df.iterrows():
    query  = "INSERT INTO user_history (user_id, session_id, session_item, artist, song, user)"
    query += "VALUES(%s, %s, %s, %s, %s, %s)"
    session.execute(query, (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, f'{row.firstName} {row.lastName}'))

In [37]:
query = "SELECT * FROM user_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['user_id', 'session_id', 'session_item', 'artist', 'song', 'user']) 

,user_id,session_id,session_item,artist,song,user
0,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,10,182,1,Three Drives,Greece 2000,Sylvie Cruz
2,10,182,2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query 3 - song_history table
The primary key for the second table will be a compound key consisting of a partition key (user) and clustering columns (user = first_name + last_name).
Records on one partition will belong to the same song and they will be sorted by user.

In [38]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query  =  "CREATE TABLE IF NOT EXISTS song_history "
query += "(song TEXT, user TEXT, \
           PRIMARY KEY(song, user))"

try:
    session.execute(query)
except Exception as e:
    print(e)     

In [39]:
# load data to first table
for i, row in df.iterrows():
    query  = "INSERT INTO song_history (song, user)"
    query += "VALUES(%s, %s)"
    session.execute(query, (row.song, f'{row.firstName} {row.lastName}'))

In [40]:
query = "SELECT * FROM song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['song', 'user']) 

,song,user
0,All Hands Against His Own,Jacqueline Lynch
1,All Hands Against His Own,Sara Johnson
2,All Hands Against His Own,Tegan Levine


### Drop the tables before closing out the sessions

In [41]:
session.execute("DROP TABLE IF EXISTS session_history")

In [42]:
session.execute("DROP TABLE IF EXISTS user_history")

In [43]:
session.execute("DROP TABLE IF EXISTS song_history")

### Close the session and cluster connection¶

In [44]:
session.shutdown()
cluster.shutdown()